# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [32]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [33]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [34]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# creating a smaller event data csv file
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [35]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [36]:
# read the csv file to a pandas data frame
df = pd.read_csv("event_datafile_new.csv")
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Develop Apache Cassandra Codes of  Project. 


### Create cluster and keyspace

#### Creating a Cluster

In [37]:
# make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # create a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [38]:
# Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS events 
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """
                   )
    
except Exception as e:
    print(e)

#### Set Keyspace

In [39]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("events")
    
except Exception as e:
    print(e)

### Create Apache Cassandra tables and run queries ###

#### 1 Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 ####

For this query, "sessionId" and "itemInSession" are considered as the composite key because the filtering condition is based on "sessionId" and "itemInSession".

In [40]:
# drop table
query = "DROP TABLE IF EXISTS eventBySessionItem"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# create table
query = "CREATE TABLE IF NOT EXISTS eventBySessionItem "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY(sessionId, itemInSession))" 
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# locate the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO eventBySessionItem (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM eventBySessionItem WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 ####

For this query, "userId", "sessionId", "itemInSession" are considered as composite key because the filtering condition is based on "userId" and "sessionId" and sorted by "itemInSession".

In [41]:
# drop table
query = "DROP TABLE IF EXISTS eventByUserSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# create table
query = "CREATE TABLE IF NOT EXISTS eventByUserSession "
query = query + "(artist text, song text, itemInSession int, userId int, sessionId int, firstName text, \
                    lastName text, PRIMARY KEY((userId, sessionId), itemInSession))" 
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
# locate the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO eventByUserSession (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
# run sql query
query = "SELECT artist, song, firstName, lastName FROM eventByUserSession WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' ####

For this query, "song" and "userId" are considered as composite key because the two elements can correspond to a unique row. 

In [42]:
# drop table
query = "DROP TABLE IF EXISTS eventBySong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# create table
query = "CREATE TABLE IF NOT EXISTS eventBySong "
query = query + "(song text, userId int, sessionId int, firstName text, \
                    lastName text, PRIMARY KEY(song, userId))" 
try:
    session.execute(query)
except Exception as e:
    print(e)    

# locate the file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO eventBySong (song, userId, sessionId, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), line[1], line[4]))
        
# run sql query
query = "SELECT firstName, lastName FROM eventBySong WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)                   

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [43]:
# Drop the tables
query = "DROP TABLE IF EXISTS eventBySessionItem"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS eventByUserSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS eventBySong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [44]:
session.shutdown()
cluster.shutdown()